# Python SQL Merge

In [1]:
""" Python SQL Merge

1. Write python code to do full load of table customer to customer_backup 
2. In sql, do some update, delete and inserts on customer table.
3. Write python code to do incremental load Using merge statement to synchronize customer_backup table
"""

' Python SQL Merge\n\n1. Write python code to do full load of table customer to customer_backup \n2. In sql, do some update, delete and inserts on customer table.\n3. Write python code to do incremental load Using merge statement to synchronize customer_backup table\n'

In [2]:
import psycopg2

In [3]:
conn = psycopg2.connect(database = "challenge1", 
    user = "postgres", 
    host= 'localhost',
    password = "password",
    port = 5432)

## Question 1

### Display Customer Table

In [4]:
def display_customer_table(conn):
    cursor = conn.cursor() 
    sql = '''SELECT * FROM customer;'''
    cursor.execute(sql) 
    results = cursor.fetchall() 
    print(results) 
    conn.commit()

display_customer_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442)), (1, 'Priyanshu', 'prynsh@gmail.com', datetime.datetime(2024, 8, 6, 15, 21, 37, 646911))]


### Create Customer Backup One Table

In [5]:
def create_customer_backup_one_table(conn):
    cursor = conn.cursor() 
    sql = '''CREATE TABLE customer_backup_one AS TABLE customer WITH NO DATA;'''
    cursor.execute(sql)
    conn.commit()

create_customer_backup_one_table(conn)

### Display Customer Backup One Table

In [6]:
def display_customer_backup_one_table(conn):
    cursor = conn.cursor() 
    sql = '''SELECT * FROM customer_backup_one;'''
    cursor.execute(sql) 
    results = cursor.fetchall() 
    print(results) 
    conn.commit()

display_customer_backup_one_table(conn)

[]


### Merge Statement to load the entire customer table in customer_backup_one in one go

In [7]:
def merge_customer_backup_one_table(conn):
    cursor = conn.cursor() 
    sql = '''MERGE INTO customer_backup_one cbo
            USING customer c ON cbo.cid = c.cid
            WHEN NOT MATCHED THEN
               INSERT (cid, name, email, lastChange)
               VALUES(c.cid, c.name, c.email, c.lastChange);'''
    cursor.execute(sql) 
    conn.commit()

merge_customer_backup_one_table(conn)
display_customer_backup_one_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (1, 'Priyanshu', 'prynsh@gmail.com', datetime.datetime(2024, 8, 6, 15, 21, 37, 646911)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442))]


## Question 2 & 3

### Using insert, update and delete trigger to incrementally update the customer_backup_one table

### Insert Trigger

In [8]:
def insert_trigger(conn):
    cursor = conn.cursor()
    sql = '''
    CREATE OR REPLACE FUNCTION insertion_trigger()
    RETURNS TRIGGER AS 
    $$
    BEGIN
        MERGE INTO customer_backup_one cbo
        USING customer c ON cbo.cid = c.cid
        WHEN NOT MATCHED THEN
           INSERT (cid, name, email, lastChange)
           VALUES(c.cid, c.name, c.email, c.lastChange);
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;'''
    cursor.execute(sql)
    conn.commit()
    cursor = conn.cursor()
    sql = '''
    CREATE TRIGGER merge_insertion_trigger
    AFTER INSERT ON customer
    FOR EACH ROW
    EXECUTE FUNCTION insertion_trigger();
    '''
    cursor.execute(sql)
    conn.commit()

insert_trigger(conn)

### Testing the above insertion trigger

In [9]:
def insert_customer_table(conn, cid, name, email):
    cursor = conn.cursor() 
    sql = '''INSERT INTO customer VALUES({}, '{}', '{}');'''.format(cid, name, email)
    cursor.execute(sql) 
    conn.commit()

In [10]:
cid = 2
name = 'Edwin'
email = 'edwin@gmail.com'
insert_customer_table(conn, cid, name, email)

In [11]:
display_customer_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442)), (1, 'Priyanshu', 'prynsh@gmail.com', datetime.datetime(2024, 8, 6, 15, 21, 37, 646911)), (2, 'Edwin', 'edwin@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 28, 885085))]


In [12]:
display_customer_backup_one_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (1, 'Priyanshu', 'prynsh@gmail.com', datetime.datetime(2024, 8, 6, 15, 21, 37, 646911)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442)), (2, 'Edwin', 'edwin@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 28, 885085))]


### Update Trigger

In [13]:
def update_trigger(conn):
    cursor = conn.cursor()
    sql = '''
    CREATE OR REPLACE FUNCTION updation_trigger()
    RETURNS TRIGGER AS 
    $$
    BEGIN
        MERGE INTO customer_backup_one cbo
        USING customer c ON cbo.cid = c.cid
        WHEN MATCHED THEN
            UPDATE SET
               cid = c.cid,
               name = c.name,
               email = c.email,
               lastChange = c.lastChange;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;'''
    cursor.execute(sql)
    conn.commit()
    cursor = conn.cursor()
    sql = '''
    CREATE TRIGGER merge_updation_trigger
    AFTER UPDATE ON customer
    FOR EACH ROW
    EXECUTE FUNCTION updation_trigger();
    '''
    cursor.execute(sql)
    conn.commit()

update_trigger(conn)

### Testing the above updation trigger

In [15]:
def update_customer_table(conn):
    cursor = conn.cursor() 
    sql = '''UPDATE customer SET email = 'priyanshu@gmail.com' WHERE cid = 1;'''
    cursor.execute(sql) 
    conn.commit()

update_customer_table(conn)

In [16]:
display_customer_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442)), (2, 'Edwin', 'edwin@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 28, 885085)), (1, 'Priyanshu', 'priyanshu@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 59, 147047))]


In [17]:
display_customer_backup_one_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (1, 'Priyanshu', 'priyanshu@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 59, 147047)), (3, 'Gabe', 'gabe@gmail.com', datetime.datetime(2024, 8, 6, 15, 20, 59, 559442)), (2, 'Edwin', 'edwin@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 28, 885085))]


### Delete Trigger

In [23]:
def delete_trigger(conn):
    cursor = conn.cursor()
    sql = '''
    CREATE OR REPLACE FUNCTION deletion_trigger()
    RETURNS TRIGGER AS 
    $$
    BEGIN
        MERGE INTO customer_backup_one cbo
        USING customer c ON cbo.cid = c.cid
        WHEN MATCHED and c.cid = OLD.cid THEN
           DELETE;
        RETURN OLD;
    END;
    $$ LANGUAGE plpgsql;'''
    cursor.execute(sql)
    conn.commit()
    cursor = conn.cursor()
    sql = '''
    CREATE TRIGGER merge_deletion_trigger
    BEFORE DELETE ON customer
    FOR EACH ROW
    EXECUTE FUNCTION deletion_trigger();
    '''
    cursor.execute(sql)
    conn.commit()

delete_trigger(conn)

In [27]:
def delete_customer_table(conn):
    cursor = conn.cursor() 
    sql = '''DELETE FROM customer WHERE cid = 2;'''
    cursor.execute(sql) 
    conn.commit()

delete_customer_table(conn)

In [28]:
display_customer_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (1, 'Priyanshu', 'priyanshu@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 59, 147047))]


In [29]:
display_customer_backup_one_table(conn)

[(4, 'Andrei', 'andrei@gmail.com', datetime.datetime(2024, 8, 6, 13, 21, 14, 869190)), (1, 'Priyanshu', 'priyanshu@gmail.com', datetime.datetime(2024, 8, 6, 15, 37, 59, 147047))]


In [30]:
conn.close()